In [ ]:
from pyspark.sql.types import FloatType, IntegerType, StringType
import pyspark.sql.functions as F
import datetime
import pandas as pd

from dateutil.parser import parse
import numpy as np
import json, os
from pyspark.storagelevel import StorageLevel

from utils import *
import warnings
warnings.filterwarnings("ignore")

flags = [False]
task = "nasi_cvr_v1"


In [ ]:

def main(day):
    dates = pd.date_range(start=day, periods=7)
    y, m, d = day.strftime("%Y"), day.strftime("%m"), day.strftime("%d")

    sql = """    
    select
        t1.*,
        cast(if(t2.affpub is not null, 1, 0) as float) as label
    from (
        select * from dsp.push_all_v1
        where year='{y}' and month='{m}' and day='{d}' and click_type='1'
    ) t1
    left join (
        SELECT * FROM dsp.rm_show_click_cvr_v2
        where aff_id = '157' and day in ({ds})
    ) t2
    on t1.uid = t2.affpub
    """.format(y=y, m=m, d=d, ds=", ".join(dates.map(lambda x: x.strftime("%Y%m%d"))))
    df = spark.sql(sql).persist(StorageLevel.MEMORY_AND_DISK)

    labels = df.select("label").toPandas()["label"]
    print("cnt>>>", day.strftime("%Y%m%d"), len(labels), labels.mean())
    if len(labels) > 1000:
        df = df.withColumn("bid_floor", F.udf(parse_float, returnType=FloatType())("bid_floor"))
            # .withColumn("modelprice", F.udf(parse_float, returnType=FloatType())("modelprice"))

        dtypes = dict(df.dtypes)
        for k, dtype in dtypes.items():
            if dtype == "string":
                df = df.withColumn(k, F.udf(fill_na)(k))

        savedPath = "/user/hive/warehouse/lixiang.db/%s/%s" % (task, day.strftime("%Y%m%d"))
        df.repartition(50).write.mode("overwrite").format("tfrecords").option("recordType", "Example").save(savedPath)

        

In [ ]:
start_date = datetime.datetime.now() - datetime.timedelta(days=14)
now = datetime.datetime.now() - datetime.timedelta(days=7)

for day in pd.date_range(start_date, now):
    dates = pd.date_range(start=day, periods=7)

    cmd = "hadoop fs -test -e /user/hive/warehouse/lixiang.db/%s/%s/_SUCCESS;echo $?" % (task, day.strftime("%Y%m%d"))
    out = exec_cmd(cmd)
    print("cmd>>>", cmd, out, dates[-1].strftime("%Y%m%d"))
    if out is not None and int(out) != 0:
        spark = set_spark(task, flags)
        sc = spark.sparkContext
        main(day)



In [ ]:
try:
    spark.stop()
except:
    pass